**Assignment 6**

**Image and Video Analytics**

**Vishali Sharma**

**21MIA1066**

Task 3: Facial Recognition to Check Fraud Cases

Objective:
Identify a suspect by comparing their facial features to a reference image to check for fraud cases, using traditional facial recognition techniques.

Task Description:


In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

**Load the reference image of the suspect and a video showing multiple faces.**

**Face Detection:**


**Use Haar Cascades to detect faces in both the reference image and the video.**

**Feature Matching:**

**Extract facial features using edge detection or geometric facial features (eye spacing, nose length, etc.).**

**Compare the features of the faces in the video with the reference face to check for a match.**
                                                                                 
**Result:**

**Output the frames where a match is found and highlight the detected face in the video.**

In [8]:
import cv2
import numpy as np
import os
from IPython.display import display, Image

# Set paths for reference image and video
reference_image_path = "suspectimg.jpg"
video_path = "suspect.mp4"

# Specify output folder for matched frames
output_folder = "suspect_frame"
os.makedirs(output_folder, exist_ok=True)

# Load the reference image and convert it to grayscale
reference_image = cv2.imread(reference_image_path)
reference_gray = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Detect face in the reference image
ref_faces = face_cascade.detectMultiScale(reference_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Ensure there's at least one face detected in the reference image
if len(ref_faces) == 0:
    print("No faces found in the reference image.")
else:
    print(f"{len(ref_faces)} face(s) detected in the reference image.")

    # Extract the detected face from the reference image (use the largest face)
    x, y, w, h = max(ref_faces, key=lambda face: face[2] * face[3])
    reference_face = reference_gray[y:y+h, x:x+w]

    # Load the video
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    # Process each frame in the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        match_found = False  # Flag to check if any match is found in the current frame

        # Process each detected face in the frame
        for (fx, fy, fw, fh) in faces:
            face_in_frame = gray_frame[fy:fy+fh, fx:fx+fw]
            resized_reference = cv2.resize(reference_face, (fw, fh))
            match_result = cv2.matchTemplate(face_in_frame, resized_reference, cv2.TM_CCOEFF_NORMED)
            _, match_val, _, _ = cv2.minMaxLoc(match_result)

            match_threshold = 0.7
            if match_val > match_threshold:
                # Draw a rectangle and label on the matching face
                cv2.rectangle(frame, (fx, fy), (fx + fw, fy + fh), (0, 255, 0), 2)
                cv2.putText(frame, f'Match: {match_val:.2f}', (fx, fy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                print(f"Match found in frame {frame_count} with similarity score: {match_val:.2f}")

                # Save the frame with the match to the output folder
                output_frame_path = os.path.join(output_folder, f'frame_{frame_count}.jpg')
                cv2.imwrite(output_frame_path, frame)
                match_found = True

        # Display matched frames in Jupyter Notebook
        if match_found:
            display(Image(filename=output_frame_path))

    # Release video capture
    cap.release()


1 face(s) detected in the reference image.


In [5]:
# Load the reference image of the suspect
suspect_image = cv2.imread('suspectimg.jpg')  # Provide your reference image file path

# Load the video file
cap = cv2.VideoCapture('suspect.mp4')  # Provide your video file path

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    # Get the total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames: {total_frames}")

# Release the video capture object when done
cap.release()


Total frames: 651


In [6]:
import cv2
from IPython.display import display, clear_output
import ipywidgets as widgets
from PIL import Image
import numpy as np

# Initialize Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load and process the suspect image
suspect_image = cv2.imread('suspectimg.jpg')
gray_suspect = cv2.cvtColor(suspect_image, cv2.COLOR_BGR2GRAY)
suspect_faces = face_cascade.detectMultiScale(gray_suspect, scaleFactor=1.1, minNeighbors=5)

# Check for faces in the suspect image
if len(suspect_faces) == 0:
    print("No face detected in the reference image.")
else:
    # Crop the first detected face in the suspect image
    x, y, w, h = suspect_faces[0]
    suspect_face = gray_suspect[y:y+h, x:x+w]

    # Use ORB (Oriented FAST and Rotated BRIEF) for feature detection and description
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(suspect_face, None)

    # Load the video
    cap = cv2.VideoCapture('suspect.mp4')
    output_widget = widgets.Output()
    display(output_widget)

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video

        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the video frame
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
        
        for (x, y, w, h) in faces:
            # Crop and analyze each detected face in the frame
            frame_face = gray_frame[y:y+h, x:x+w]

            # Detect features in the face from the current frame
            keypoints2, descriptors2 = orb.detectAndCompute(frame_face, None)

            # Ensure both faces have descriptors for matching
            if descriptors1 is not None and descriptors2 is not None:
                # Use BFMatcher with Hamming distance
                bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                matches = bf.match(descriptors1, descriptors2)

                # Filter good matches based on distance
                good_matches = [m for m in matches if m.distance < 30]

                # Check if the number of good matches exceeds a threshold
                if len(good_matches) > 15:
                    # Draw rectangle and label the suspect face
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, "Suspect Person", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Display the processed frame in the Jupyter Notebook
        with output_widget:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(frame_rgb)
            clear_output(wait=True)
            display(img_pil)

        # Optionally display the frame in a separate window
        cv2.imshow("Video Frame - Suspect Detection", frame)

        # Press 'q' to exit early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release video and close windows
    cap.release()
    cv2.destroyAllWindows()


Output()

In [17]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Convert the reference image to grayscale
gray_suspect = cv2.cvtColor(suspect_image, cv2.COLOR_BGR2GRAY)

# Detect faces in the reference image
suspect_faces = face_cascade.detectMultiScale(gray_suspect, scaleFactor=1.1, minNeighbors=5)

# Check if any faces were detected in the reference image
if len(suspect_faces) == 0:
    print("No face detected in the reference image.")
else:
    # Crop and display each detected face in the reference image
    for (x, y, w, h) in suspect_faces:
        suspect_face = gray_suspect[y:y+h, x:x+w]
        cv2.imshow("Suspect Face", suspect_face)
        cv2.waitKey(0)  # Press any key to close the suspect face window

# Now process each frame in the video to detect faces
while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break  # Exit the loop if no frame is read (end of video)

    # Convert the frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the current video frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    # Draw rectangles around detected faces in the frame
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the frame with detected faces
    cv2.imshow("Video Frame - Face Detection", frame)

    # Press 'q' to quit early
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()
